# Tasks

## P 3.1: Download the CarSharing dataset from Canvas and apply the normal linear regression and the Ridge regression methods to predict the 'demand' column. Use 5 different values for the regularisation parameter in the Ridge regression and repeat the prediction. Calculate all evaluation metrics discussed in the lecture session for both models and all repetitions and print them. Interpret the results and report your interpretation in the report cell (completing the report cell is required) (15%)

In [ ]:
############# WRITE YOUR CODE IN THIS CELL (IF APPLICABLE)  ####################
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
data = pd.read_csv("/content/CarSharing.csv")
data.drop(columns=["id", "timestamp"], inplace=True)
data


FileNotFoundError: [Errno 2] No such file or directory: '/content/CarSharing.csv'

In [ ]:
#data prepration
data.info()
data.describe()
data.head()
print(data.isnull().sum())

# filling null with mean value for number and mode for string type of data
data.fillna({
    "temp" : data["temp"].mean(),
    "temp_feel": data["temp_feel"].mean(),
    "windspeed" : data["windspeed"].mean(),
    "humidity" : data['humidity'].mean()
}, inplace=True)
print(data.isnull().sum())
print(data.isnull())
data.head()

In [ ]:
# converting categorical values into numberical value using pandas get_dummies()
df_encoded = pd.get_dummies(data, columns=['season', 'holiday', 'workingday', 'weather'], drop_first=True)
df_encoded.head()

In [ ]:
df_encoded

In [ ]:

scaler = StandardScaler()
numerical_cols = ['temp', 'temp_feel', 'humidity', 'windspeed']
df_encoded[numerical_cols] = scaler.fit_transform(df_encoded[numerical_cols])

X = df_encoded.drop('demand', axis=1)
y = df_encoded['demand']
#converting data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge.fit(X_train,y_train)

In [ ]:
y_pred_ridge1= ridge.predict(X_test)

In [ ]:
y_pred = model.predict(X_test)
print(y_pred)

In [ ]:

from sklearn.metrics import mean_squared_error, r2_score

# Calculate and print R^2 score.
r2 = r2_score(y_test, y_pred)
print(f"R-squared normal regression: {r2:.4f}")

In [ ]:
#Evaluating ridge model performance using the coefficient of determination is easy with scikit-learn.

from sklearn.metrics import mean_squared_error, r2_score

# Calculate and print R^2 score.
r2 = r2_score(y_test, y_pred_ridge1)
print(f"R-squared ridge: {r2:.4f}")